In [3]:
import json
import pickle

In [4]:
# define desired keys namess

element_names = ['regkey_opened','regkey_read','regkey_written','regkey_deleted','file_created',
        'file_opened','file_read','file_deleted', 'directory_created','dll_loaded','apistats','dropped','strings']

In [8]:
# explore the key name
with open("../dataset/mali/report1.json") as fr:
    data = json.load(fr)

data.keys()

dict_keys(['info', 'procmemory', 'target', 'buffer', 'network', 'signatures', 'static', 'dropped', 'behavior', 'debug', 'strings', 'metadata'])

In [5]:
def item_generator(json_input, lookup_key):
    '''
        list all the key values inside a json file
    '''
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield v
            else:
                yield from item_generator(v, lookup_key)
    elif isinstance(json_input, list):
        for item in json_input:
            yield from item_generator(item, lookup_key)

In [6]:
# value_list = []
# for item in item_generator(data, "regkey_opened"):
#     print(item)
#     value_list.append(item)

# # value_list

In [11]:
# test on special key values
for item in item_generator(data, "apistats"):
    for id in item:   
        print(id) 
        # print(item[id])

for items in item_generator(data, "dropped"):
    for item in items:   
        try:
            print(item['name']) 
        except:
            print(item['basename'])
        # print(item[id])

3972
3836
3284
3732
4052
0102db0fb8cc346e_newt@advertising[1].txt
0484594317d59053_bbxr2bu[1].png
07c2dfabf1783aa3_aacl6lf[1].png
0a1982ebe9c7b686_www.msn[1].xml
0a43c508aa770e79_bbvwnms[1].jpg
0bc0421b3c0f5028_bbzlvob[1].jpg
0c4bb25724b0fcd7_bbzniwo[1].jpg
0e5e25462d90e1cc_bbzex4u[1].jpg
0fc31506e968c1cf_bbznbmn[1].jpg
103300c4e6aeed2f_bbz4p7l[1].jpg
1033149232f876f9_bbzn5gt[1].jpg
10775405b4fb01f7_bby5v95[1].jpg
14324c7c794a2119_bbznch7[1].jpg
14a5e69048b67ca1_bbzlic3[1].jpg
1657b76becc2ec2c_6bada8974a10c4bd62cc921d13e43b18_88614ffad35d353421b8a7e1fe18fce4
168c6e5c0d479cd9_bbsu4ri[1].png
17fdf7a0a664ba0e_bbznem1[1].jpg
18c65926d221ec49_bbzmmve[1].png
198adacab248143a_bbznaym[1].png
19ecf97d62bdda39_bbzo5dz[1].jpg
1b38622f12492ca3_aamir2o[1].png
1d17f48267ecd278_aa60arv[1].png
1d6e356ed275ad94_07cef2f654e3ed6050ffc9b6eb844250_bacc6cd2b29f18349081c9fd2343833b
1f027a9e4bd073ae_aa3ltd8[1].png
1fcdca2663bfc591_bbaj56p[1].png
1fdb650445d104f1_aagpnha[1].png
208009ce5b16b284_bbznoww[1].jpg


In [12]:
from pathlib import Path

# recursively build the dict for feature values
feature_dict = {
    'ID': [],
    'Label': [],
}
# initialize the element to blank list
for element in element_names:
    feature_dict[element] = []

mali_path = '../dataset/mali'
benign_path = '../dataset/benign'

for path in [mali_path, benign_path]:
    ID = 0
    path = Path(path)
    if path.stem == 'mali':
        label = 1
    else:
        label = 0
    for file in path.rglob("*.json"):
        try:
            with file.open("r", encoding='utf-8') as fr:
                data = json.load(fr)
                feature_dict['ID'].append(ID)
                feature_dict['Label'].append(label)
                for element in element_names:
                    item_list = []
                    # special case to process
                    if element == 'apistats':
                        for item in item_generator(data, element):
                            # there are normal multiple ids inside it
                            for id in item:   
                                # extend list type
                                item_list.extend(item[id])
                    elif element == 'dropped':
                        for items in item_generator(data, "dropped"):
                            for item in items:   
                                try:
                                    # append str type 
                                    item_list.append(item['name']) 
                                except:
                                    item_list.append(item['basename'])
                    else:
                        for item in item_generator(data, element):
                            item_list.extend(item)

                    feature_dict[element].append(item_list)
                
                ID += 1
        except Exception as e:
            print(e)


Expecting ':' delimiter: line 908002 column 30 (char 61212619)
Expecting ':' delimiter: line 54491 column 32 (char 3414594)
Expecting value: line 16372 column 34 (char 915173)
Expecting value: line 316630 column 41 (char 14177507)
Expecting property name enclosed in double quotes: line 605342 column 29 (char 26541214)
Expecting value: line 1095827 column 33 (char 48003467)
Expecting ',' delimiter: line 470906 column 50 (char 20679853)
Expecting ',' delimiter: line 863953 column 22 (char 42832086)
Expecting property name enclosed in double quotes: line 229633 column 25 (char 10084493)
Expecting property name enclosed in double quotes: line 317553 column 25 (char 13834725)
Expecting ':' delimiter: line 4034254 column 31 (char 176305224)
Expecting ',' delimiter: line 1449391 column 56 (char 63517875)


In [13]:
import dask.dataframe as dd
import pandas as pd

fea_df = dd.from_pandas(pd.DataFrame.from_dict(feature_dict, orient='columns'), npartitions=2)

In [14]:
fea_df.compute()
# fea_df.to_parquet('raw_dict.parquet',compression="snappy", engine="pyarrow")

,ID,Label,regkey_opened,regkey_read,regkey_written,regkey_deleted,file_created,file_opened,file_read,file_deleted,directory_created,dll_loaded,apistats,dropped,strings
0,0,1,[HKEY_LOCAL_MACHINE\System\CurrentControlSet\S...,[HKEY_LOCAL_MACHINE\SOFTWARE\MICROSOFT\Cryptog...,[HKEY_CURRENT_USER\Software\Temp\{F83C114F-4AD...,[],[],[C:\Users\newt\AppData\Local\Temp\VirusShare_0...,[C:\Users\newt\AppData\Local\Temp\VirusShare_0...,[C:\Users\newt\AppData\Local\Temp\VirusShare_0...,"[C:\Users\newt\AppData\Local\Temp, C:\Users\ne...","[ntmarta.dll, dwmapi.dll, urlmon.dll, CRYPTSP....","[CreateToolhelp32Snapshot, NtDuplicateObject, ...",[],"[!This program cannot be run in DOS mode., .rd..."
1,1,1,[HKEY_CLASSES_ROOT\PROTOCOLS\Name-Space Handle...,[HKEY_LOCAL_MACHINE\SOFTWARE\Wow6432Node\Micro...,[HKEY_CURRENT_USER\Software\Microsoft\Internet...,[HKEY_LOCAL_MACHINE\SOFTWARE\Wow6432Node\Micro...,[C:\Users\newt\AppData\Local\Microsoft\Windows...,"[C:\, C:\Users\newt\AppData\LocalLow\Microsoft...",[C:\Users\newt\AppData\LocalLow\Microsoft\Cryp...,"[C:\Users\newt\AppData\Local\Temp\Tar6518.tmp,...",[C:\Users\newt\AppData\Local\Microsoft\Windows...,"[C:\Windows\system32\kernel32.dll, API-MS-Win-...","[NtOpenSection, GetVolumePathNameW, WSARecv, g...","[0102db0fb8cc346e_newt@advertising[1].txt, 048...","[ARQCAAAAAADAAAAAAAAARg==, TAAAAA==, !This pro..."
2,2,1,[],[],[],[],[],[],[],[],[],[],[NtTerminateProcess],[],"[!This program cannot be run in DOS mode., Ric..."
3,3,1,[],[],[],[],[],[],[],[],[],[],[],[],"[!This program cannot be run in DOS mode., `.r..."
4,4,1,[],[],[],[],[],[],[],[],[],[],[NtTerminateProcess],[],"[!This program cannot be run in DOS mode., Ric..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,137,0,[HKEY_LOCAL_MACHINE\SOFTWARE\Justsystem\ATOK\S...,[HKEY_LOCAL_MACHINE\SOFTWARE\Wow6432Node\Micro...,"[HKEY_CURRENT_USER\.bin\(Default), HKEY_CURREN...",[],[],[C:\Users\newt\AppData\Roaming\Adobe\Headlight...,"[C:\Users\newt\Favorites\desktop.ini, C:\Users...",[C:\Users\newt\AppData\Local\Microsoft\Windows...,[C:\Users\newt\AppData\Roaming\Adobe\Headlight...,"[ADVAPI32.dll, ntmarta.dll, API-MS-Win-Securit...","[RegCreateKeyExW, NtDuplicateObject, NtOpenSec...",[],[V3JpdGVGaWxl]
317,138,0,[HKEY_LOCAL_MACHINE\Software\Clients\Media\Win...,[HKEY_LOCAL_MACHINE\SOFTWARE\Classes\.bin\DocO...,[HKEY_CURRENT_USER\Local Settings\Software\Mic...,[],[],"[C:\Windows\AppPatch\sysmain.sdb, C:\, C:\Prog...","[C:\Windows\System32\EhStorShell.dll, C:\Windo...",[C:\Users\newt\AppData\Local\Microsoft\Windows...,[C:\Users\newt\AppData\Local\Microsoft\Windows...,"[C:\Windows\system32\ntshrui.dll, ADVAPI32.dll...","[RegCreateKeyExW, NtDuplicateObject, NtOpenSec...",[],"[, /4B5', y.a879, ~XDI$`, 6EI9{"", 7B6 6, <&LU..."
318,139,0,[HKEY_LOCAL_MACHINE\Software\Microsoft\Windows...,[HKEY_LOCAL_MACHINE\SOFTWARE\Classes\Wow6432No...,"[HKEY_CURRENT_USER\.bin\(Default), HKEY_CURREN...",[],[],"[C:\Users\newt\AppData\Local, C:\Windows\AppPa...","[C:\Users\newt\Favorites\desktop.ini, C:\Users...",[C:\Users\newt\AppData\Local\Microsoft\Windows...,[C:\Users\newt\AppData\Local\Microsoft\Windows...,"[C:\Windows\system32\ntshrui.dll, netutils.dll...","[RegCreateKeyExW, NtDuplicateObject, NtOpenSec...",[],"[V3JpdGVGaWxl, W[pe/O, ?Q*N(c, ceibLv, wm4t1r,..."
319,140,0,"[HKEY_LOCAL_MACHINE\Software\Microsoft\Rpc, HK...",[HKEY_LOCAL_MACHINE\SOFTWARE\MICROSOFT\Rpc\Max...,[],[],[],[],[],[],[],"[kernel32, api-ms-win-core-fibers-l1-1-1, api-...","[LdrUnloadDll, RegCloseKey, NtDuplicateObject,...",[],"[!This program cannot be run in DOS mode., P|R..."


In [16]:
with open('../dataset/features/raw_features.pickle', 'wb') as handle:
    pickle.dump(fea_df, handle, protocol=pickle.HIGHEST_PROTOCOL)